In [1]:
import pyodbc
import requests
import pandas as pd
import numpy as np
import json
import datetime

I want to make an API call that returns values from which I can extract 2 different DFs. \
The DataFrames I get are: \
1.  Country data: name, currency, and currency symbol
2.  Exchange data: I obtain the available information from the exchanges in each country.

The relationship between the two of them is the country column that will be from 1 to many, \
since there is more than one stock exchange per country, such as in the USA.

In [ ]:
# An API call to get the exchanges that are recognied
param = {   'access_key': 'a9c154cf6e91a6e1f65e1993f296b253' }
key = 'a9c154cf6e91a6e1f65e1993f296b253'

exchanges= f'http://api.marketstack.com/v1/exchanges?access_key={key}'

exchnage_response = requests.get(exchanges,param)
exch_result = exchnage_response.json()
exch_result = exch_result['data']

exch_df = pd.DataFrame(exch_result)
exch_df = exch_df[['name','acronym','mic','country','country_code','city','website']] 

exch_df.head()

In [ ]:
# Creating the array, where i will keep the info from the last API call that solve me the problem of the data country I want to keep in another DF
country_data = []
for elem in exch_result:
  try:
    # There is a posibility that not all the elems in the array 'exch_result' will have values for all the params, so I have to put the try catch
    country = elem['country']  
    code = elem['currency']['code']
    symbol = elem['currency']['symbol']
    country_data.append({'country':country,'code':code,'symbol':symbol})
  except: 
    pass


countries_df = pd.DataFrame(country_data)
countries_df.drop_duplicates(inplace=True)
print(countries_df.shape)
countries_df.head()

**Connversion of the API call into a DF**
To convert the response that the API gives us, what we have done is the following. \
1. Convert the response into a txt (so I can see what is returned). \
2. Convert the array into JSON format
3. Since it returns an array, I only want to keep the element that contains the information, therefore I choose the second element.
4. Convert that array that it returns to me into a DF with the pandas library

**A very short explanation of every value**

*   Date: The date of the quote.
*   Symbol: The symbol of the action.
*   Stock Exchange: The stock exchange on which the stock is listed.
*   Open: The opening price of the stock.
*   High: The maximum price reached by the stock during the day.
*   Low: The minimum price reached by the stock during the day.
*   Close: The closing price of the stock.
*   Volume: The number of shares traded during the day.


In [10]:
def api_call (symbols,date_from,date_to): 
  param = {'access_key': 'a9c154cf6e91a6e1f65e1993f296b253', 'limit':'1000'}
  key = 'a9c154cf6e91a6e1f65e1993f296b253'
  
  symbols_str = ','.join(symbols)

  url = f'http://api.marketstack.com/v1/eod?access_key={key}&symbols={symbols_str}& date_from={date_from}& date_to={date_to}'

  response = requests.get(url,params=param)
  
  return response.json()


In [3]:
def create_df(data):
  # Creating the DF and taking the columns I am interested on 
  df = pd.DataFrame(data)
  df = df[['symbol','exchange','date','open','close','high','low','volume']]
  
  # Reformating the date column into the format in want to keep
  df['date'] = pd.to_datetime(df['date'])
  df['date'] = df['date'].dt.strftime('%Y-%m-%d')

  return df

In [13]:
date_to = datetime.datetime.today()
date_from = date_to - datetime.timedelta(days=90)

date_to = date_to.strftime('%Y-%m-%d')
date_from = date_from.strftime('%Y-%m-%d')

# Importante a tener en cuenta, es que para asegurarme que la accion esta bien 
symbols_to_track = ['BBVA','SAN','IBE.BMEX','ANA.BMEX','AENA.BMEX','GOOGL','AMZN','META','MSFT','TSLA',
                    'BMW.XFRA','ADS.XSTU','SIE.XFRA','AMXL.XMEX','BP.XLON','SHEL.XLON','VOD.XLON']

# Call the function to get the companies's data
# Because the API call return an array with more than one value, I only keep the relevant data. 
data_for_df = api_call(date_from = date_from, date_to = date_to, symbols= symbols_to_track)['data']

# Call the function to create the DF with the values I want to keep
stock_data_df = create_df(data_for_df)

print('A bit information of the df: ', stock_data_df.shape,'\n',)
print(f"Those are the unique values of the stock market action ({len(stock_data_df['symbol'].unique())})", stock_data_df['symbol'].unique(),'\n',)
print('A comparassion between the amount of actions I ask and the amount I get is the same ? ', len(symbols_to_track)== len(stock_data_df['symbol'].unique()))
stock_data_df.head()


A bit information of the df:  (910, 8) 

Those are the unique values of the stock market action (16) ['ADS.XSTU' 'AENA.BMEX' 'AMZN' 'ANA.BMEX' 'BBVA' 'BMW.XFRA' 'GOOGL'
 'IBE.BMEX' 'META' 'MSFT' 'SAN' 'SIE.XFRA' 'TSLA' 'BP.XLON' 'SHEL.XLON'
 'VOD.XLON'] 

A comparassion between the amount of actions I ask and the amount I get is the same ?  False


,symbol,exchange,date,open,close,high,low,volume
0,ADS.XSTU,XSTU,2023-11-24,184.84,184.18,185.68,183.34,326.0
1,AENA.BMEX,BMEX,2023-11-24,155.80,156.60,157.50,155.35,92332.0
2,AMZN,XNAS,2023-11-24,146.70,146.74,147.20,145.32,22378400.0
3,ANA.BMEX,BMEX,2023-11-24,125.25,123.85,125.75,123.20,98378.0
4,BBVA,XNYS,2023-11-24,9.19,9.12,9.22,9.12,774000.0


In [14]:
stock_data_df['date'].unique()

array(['2023-11-24', '2023-11-23', '2023-11-22', '2023-11-21',
       '2023-11-20', '2023-11-17', '2023-11-16', '2023-11-15',
       '2023-11-14', '2023-11-13', '2023-11-10', '2023-11-09',
       '2023-11-08', '2023-11-07', '2023-11-06', '2023-11-03',
       '2023-11-02', '2023-11-01', '2023-10-31', '2023-10-30',
       '2023-10-27', '2023-10-26', '2023-10-25', '2023-10-24',
       '2023-10-23', '2023-10-20', '2023-10-19', '2023-10-18',
       '2023-10-17', '2023-10-16', '2023-10-13', '2023-10-12',
       '2023-10-11', '2023-10-10', '2023-10-09', '2023-10-06',
       '2023-10-05', '2023-10-04', '2023-10-03', '2023-10-02',
       '2023-09-29', '2023-09-28', '2023-09-27', '2023-09-26',
       '2023-09-25', '2023-09-22', '2023-09-21', '2023-09-20',
       '2023-09-19', '2023-09-18', '2023-09-15', '2023-09-14',
       '2023-09-13', '2023-09-12', '2023-09-11', '2023-09-08',
       '2023-09-07', '2023-09-06', '2023-09-05', '2023-09-04',
       '2023-09-01', '2023-08-31', '2023-08-30', '2023-

In [ ]:
# exch_df.head()
# countries_df.head()
exch_df.shape

In [15]:
# Function to connect with SQL SERVER and return the cursor to execute the queries
def connection_sql(close=False):
    
        # Configura la conexión a SQL Server
        server = 'DESKTOP-6AV10PA'
        database = 'Test'
        username = 'yitzhak'
        password = 'admin123'
        driver = '{SQL Server}' 

        # Open the connection
        connection_string = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()

        if close:
                conn.close()
                
        return cursor

In [ ]:
# Insert values into EXCHANGE TABLE
cursor = connection_sql()

columns = ','.join(exch_df.columns)
values = '?,'*(len(exch_df.columns)-1) + '?'
query = f"INSERT INTO [Test].[dbo].[exchanges] ({columns}) VALUES ({values})"


for index,row in exch_df.iterrows():

  try:
    cursor.execute(query,(row['name'],row['acronym'],row['mic'],row['country'],row['country_code'],row['city'],row['website']))
    cursor.commit()
    
  except Exception as error:
    print('Algo va mal, ' , error,'\n',)
    print('Estos son los valores que queremos insertar: \n',
          'name ->' ,  row['name'], '\n',
          'acronym ->',row['acronym'],'\n',
          'mic ->', row['mic'],'\n',
          'country ->',row['country'],'\n',
          'country_code ->',row['country_code'],'\n',
          'city ->',row['city'],'\n',
          'website ->',row['website'])
    break
   
# Close the connection with SQL
connection_sql(close=True)

In [ ]:
# Insert values into COUNTRY TABLE
cursor = connection_sql()

columns = ','.join(countries_df.columns)
values = '?,'*(len(countries_df.columns)-1) + '?'
query = f"INSERT INTO [Test].[dbo].[countries] ({columns}) VALUES ({values})"


for index,row in countries_df.iterrows():

  try:
    cursor.execute(query,(row['country'],row['code'],row['symbol']))
    cursor.commit()
    
  except Exception as error:
    print('Algo va mal, ' , error,'\n',)
    print('Estos son los valores que queremos insertar: \n',
          'country ->',row['country'],'\n',
          'country_code ->',row['code'],'\n',
          'city ->',row['symbol'],'\n',
          )
    break
   
# Close the connection with SQL
connection_sql(close=True)

In [17]:
# Insert values into Stock Market
cursor = connection_sql()

columns = ','.join(stock_data_df.columns)
values = '?,'*(len(stock_data_df.columns)-1) + '?'
query = f"INSERT INTO [Test].[dbo].[Stock_Markert] ({columns}) VALUES ({values})"


for index,row in stock_data_df.iterrows():

  try:
    
    # print(f"INSERT INTO [Test].[dbo].[Stock_Markert] ([symbol],[exchange],[date],[open],[close],[high],[low],[volume]) \
    #        VALUES ('{row['symbol']}','{row['exchange']}','{row['date']}',{row['open']},{row['close']},{row['high']},{row['low']},{row['volume']})")
    cursor.execute(f"INSERT INTO [Test].[dbo].[Stock_Markert] ([symbol],[exchange],[date],[open],[close],[high],[low],[volume]) \
           VALUES ('{row['symbol']}','{row['exchange']}','{row['date']}',{row['open']},{row['close']},{row['high']},{row['low']},{row['volume']})")
    cursor.commit()

  except Exception as error:
    print('Algo va mal \n' , error,'\n',)
    print('Estos son los valores que queremos insertar: \n',
          'symbol ->' ,  row['symbol'], '\n',
          'exchange ->',row['exchange'],'\n',
          'date ->', row['date'],'\n',
          'open ->',row['open'],'\n',
          'close ->',row['close'],'\n',
          'high ->',row['high'],'\n',
          'low ->',row['low'],'\n',
          'volume ->',row['volume'])
    break
   
# Close the connection with SQL
connection_sql(close=True)